In [1]:
import pandas as pd
import numpy as np
from pandas_datareader import DataReader

import datetime
import requests
import bs4 as bs
import pickle
#from pandas_datareader import data

<h2>Downloading of data</h2>

In [2]:
stocks_list = []

In [3]:
def save_sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[1].text
        tickers.append(ticker)
        
    with open("sp500tickers.pickle","wb") as f:
        pickle.dump(tickers,f)
        
    return tickers

stocks = save_sp500_tickers()

In [4]:
# Replace dots with hyphen to correct incompatibility between Wikipedia and Yahoo! Finance API

stocks = [stock.replace('.', '-') for stock in stocks]
stocks.append('^GSPC')

In [5]:
stocks

['3M Company',
 'Abbott Laboratories',
 'AbbVie Inc-',
 'ABIOMED Inc',
 'Accenture plc',
 'Activision Blizzard',
 'Adobe Systems Inc',
 'Advanced Micro Devices Inc',
 'Advance Auto Parts',
 'AES Corp',
 'Affiliated Managers Group Inc',
 'AFLAC Inc',
 'Agilent Technologies Inc',
 'Air Products & Chemicals Inc',
 'Akamai Technologies Inc',
 'Alaska Air Group Inc',
 'Albemarle Corp',
 'Alexandria Real Estate Equities',
 'Alexion Pharmaceuticals',
 'Align Technology',
 'Allegion',
 'Allergan, Plc',
 'Alliance Data Systems',
 'Alliant Energy Corp',
 'Allstate Corp',
 'Alphabet Inc Class A',
 'Alphabet Inc Class C',
 'Altria Group Inc',
 'Amazon-com Inc-',
 'Ameren Corp',
 'American Airlines Group',
 'American Electric Power',
 'American Express Co',
 'American International Group',
 'American Tower Corp-',
 'American Water Works Company Inc',
 'Ameriprise Financial',
 'AmerisourceBergen Corp',
 'AMETEK Inc-',
 'Amgen Inc-',
 'Amphenol Corp',
 'Anadarko Petroleum Corp',
 'Analog Devices, Inc

In [6]:
VIX = DataReader("^VIX",  "yahoo", datetime(2016,1,1), datetime(2016,4,1))

TypeError: 'module' object is not callable

In [8]:
# We would like all available data from 03/01/2007 until today.
start_date = '1900-01-01'
end = datetime.date.today()

def get_stocks_data(ticker, source, start_date, end):
    # User pandas_reader.data.DataReader to load the desired data. As simple as that.
    Ticker = DataReader(ticker, source, start_date, end)
    Ticker['Ticker'] = ticker
    return Ticker
    
stocks_data = [get_stocks_data(ticker, 'yahoo', start_date, end) for ticker in stocks]

OverflowError: mktime argument out of range

In [ ]:
stocks_data = pd.concat(stocks_data, axis=0)
stocks_data = stocks_data.set_index(['Ticker'], append=True)
stocks_data.head()

In [8]:
dfcsv = pd.read_csv('Data/F-F_Research_Data_5_Factors_2x3_daily.csv', header=2, index_col = 0)
dfcsv

,Mkt-RF,SMB,HML,RMW,CMA,RF
19630701,-0.67,0.00,-0.32,-0.01,0.15,0.012
19630702,0.79,-0.27,0.27,-0.07,-0.19,0.012
19630703,0.63,-0.17,-0.09,0.17,-0.33,0.012
19630705,0.40,0.08,-0.28,0.08,-0.33,0.012
19630708,-0.63,0.04,-0.18,-0.29,0.13,0.012
19630709,0.45,0.00,0.10,0.14,-0.04,0.012
19630710,-0.18,0.21,0.01,0.06,-0.07,0.012
19630711,-0.16,0.14,-0.30,-0.06,0.05,0.012
19630712,-0.12,0.02,-0.11,0.12,0.04,0.012
19630715,-0.62,0.07,-0.03,0.17,-0.06,0.012


<h2>Question1</h2>

<h2>Question2</h2>

<h2>Question3</h2>

<h2>Question4</h2>